# Video Sampling Example

This notebook demonstrates how to use the sampling functions to generate videos from text prompts.

In [1]:
import sys
from sample import sample_videos, sample_single_video, prompts
from release_server import load_merge_config, load_all, compile_models

SageAttention loaded successfully
flash attn 2 available False
flash attn 3 available False
sage attn available True
SAGEATTN_AVAILABLE: True
DO_COMPILE False


In [2]:
import torch
import gc
config = load_merge_config("configs/self_forcing_server_14b.yaml")
models = load_all(config)
compile_models(models)
print("✅ Models loaded and ready to use!")
gc.collect()
torch.cuda.empty_cache()


2025-10-20 07:06:05.601 - INFO - Starting model loading...
2025-10-20 07:06:05.603 - DEBUG - Using checkpoint: checkpoints/krea-realtime-video-14b.safetensors
Loading transformer:   0%|          | 0/4 [00:00<?, ?it/s]2025-10-20 07:06:07.864 - DEBUG - Transformer import took: 2.26s
2025-10-20 07:06:15.663 - DEBUG - Loading transformer state dict from checkpoints/krea-realtime-video-14b.safetensors


Loading WAN model, with name Wan2.1-T2V-14B
timestep shift,  5.0


2025-10-20 07:06:52.198 - DEBUG - Transformer load completed in: 44.33s, total: 46.59s
2025-10-20 07:06:52.200 - DEBUG - Loading transformer took: 46.59s
Loading text encoder:  25%|██▌       | 1/4 [00:46<02:19, 46.59s/it]2025-10-20 07:06:52.203 - DEBUG - Text encoder import took: 0.00s
2025-10-20 07:06:55.298 - DEBUG - Text encoder load completed in: 3.09s, total: 3.09s
2025-10-20 07:06:55.300 - DEBUG - Loading text encoder took: 3.10s
Loading VAE:  50%|█████     | 2/4 [00:49<00:42, 21.01s/it]         2025-10-20 07:06:55.528 - DEBUG - Using demo_utils.vae_block3.VAEEncoderWrapper
2025-10-20 07:06:55.529 - DEBUG - Using demo_utils.vae_block3.VAEDecoderWrapper
2025-10-20 07:06:55.561 - INFO - loading Wan-2.1/Wan2.1-T2V-1.3B/Wan2.1_VAE.pth
2025-10-20 07:06:56.646 - DEBUG - VAE load completed in: 1.12s
2025-10-20 07:06:56.678 - DEBUG - Loading VAE took: 1.15s
Initializing pipeline:  75%|███████▌  | 3/4 [00:51<00:12, 12.05s/it]2025-10-20 07:06:56.684 - DEBUG - Pipeline import took: 0.00s
20

Incompatible _IncompatibleKeys(missing_keys=['mean', 'std'], unexpected_keys=[]) while loading vae decoder
KV inference with 3 frames per block
✅ Models loaded and ready to use!


In [6]:
import sample
from sample import GenerateParams
params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=8,
    kv_cache_num_frames=3,
    do_kv_recomp=True,
    num_denoising_steps=4,
    seed=123,
)
prompts = [
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The camera rotates around the samurai, as he swings",
]
results = sample.sample_videos(
    prompts_list=prompts,
    models=models,
    params=params,
    output_dir="outputs",
    save_videos=True,
    fps=16
)

Sampling videos:   0%|          | 0/1 [00:00<?, ?it/s]


📝 Prompt 1/1: A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The camera rotates around the samurai, as he swings
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([1000.0000,  908.8427,  713.9794,    0.0000], device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache
Zero reinitialization of kv cache


Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
✅ Generated 90 frames


/vast/erwann/realtime-video/.venv/lib/python3.11/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
Sampling videos: 100%|██████████| 1/1 [00:15<00:00, 15.96s/it]

✅ Video saved to outputs/prompt_000.mp4

🎉 All videos generated successfully!


## Video to Video

In [ ]:

params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=9,
    seed=1234,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=4,

    input_vid = "https://cdn-uploads.huggingface.co/production/uploads/62a2712903bf94c3ac3ae004/UjM1G0hthNGdtItaE-qGc.mp4" # ADD this
)
prompts = [
    "a cat knight is riding a horse across a dusty plain. the cat is a black cat wearing armor",
]
results = sample.sample_videos(
    prompts_list=[p],
    models=models,  # Reuse loaded models
    params=params,
    output_dir="out_release",
    save_videos=True,
    fps=16
)